In [1]:
# pip install --upgrade kaggle-environments

In [2]:
# # imports
# !pip install stable-baselines3 pyglet imageio-ffmpeg gymnasium[atari,accept-rom-license] ale-py > /dev/null 2>&1
# !pip install gymnasium==1.0.0
!pip install gymnasium==0.29.1
!pip install ale_py
!pip install stable-baselines3
# !pip install stable-baselines3[extra]
# pip install sb3-contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.9 requires gymnasium==0.29.0, but you have gymnasium 0.29.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.1 MB/s eta 0:00:00a 0:00:01


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import ale_py

In [5]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import torch
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback

from gymnasium.wrappers import ResizeObservation

In [6]:
print(gym.__version__)

0.29.1


In [7]:
from stable_baselines3.common.monitor import Monitor

In [8]:
import matplotlib.pyplot as plt

In [9]:
import os

In [10]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CheckpointCallback

In [11]:
gym.register_envs(ale_py)

In [12]:
# from sb3_contrib import QRDQN

In [13]:
from datetime import datetime
from stable_baselines3 import A2C
from stable_baselines3.ppo.policies import MlpPolicy
from wandb.integration.sb3 import WandbCallback

In [14]:
from stable_baselines3.common.env_util import make_vec_env

In [15]:
from stable_baselines3 import PPO

In [16]:
from stable_baselines3.common.callbacks import EvalCallback, CallbackList

In [17]:
import collections

In [18]:
from stable_baselines3.common.callbacks import BaseCallback
import torch

In [19]:
from stable_baselines3.common.vec_env import VecEnvWrapper

In [20]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [21]:
# configuration file
config = {
    "policy_type": "CnnPolicy",
    "total_timesteps": 35000000, # 1000000, 3000000, 20000000
    "Algo": "PPO",
    "env_name": "ALE/DonkeyKong-v5",
    "model_name": "ALE/DonkeyKong-v5",
    "export_path": "/kaggle/working/exports/",
    "videos_path": "/kaggle/working/videos/",
}



In [22]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY ")

In [23]:
# Wandb setup
wandb.login(key=secret_value_0)
run = wandb.init(
    project="Project_1",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    save_code=True,  # optional
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amrit0081 (education_ai). Use `wandb login --relogin` to force relogin


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [24]:
class MyVecTransposeImage(VecEnvWrapper):
    def __init__(self, venv, skip=False):
        super().__init__(venv)
        self.skip = skip

        # Get original shape: e.g., (84, 84, 4)
        old_shape = self.observation_space.shape
        # Transpose shape to (C, H, W)
        new_shape = (old_shape[2], old_shape[0], old_shape[1])  # (4, 84, 84)

        # Use the original low/high if they are uniform; if not, use min/max appropriately
        low_val = self.observation_space.low.min()
        high_val = self.observation_space.high.max()

        self.observation_space = gym.spaces.Box(
            low=low_val,
            high=high_val,
            shape=new_shape,
            dtype=self.observation_space.dtype
        )

    def reset(self):
        obs = self.venv.reset()
        return self.transpose_observations(obs)

    def step_async(self, actions):
        self.venv.step_async(actions)

    def step_wait(self):
        obs, rewards, dones, infos = self.venv.step_wait()
        return self.transpose_observations(obs), rewards, dones, infos

    def transpose_observations(self, obs):
        if self.skip:
            return obs
        if isinstance(obs, dict):
            for key, val in obs.items():
                obs[key] = self._transpose(val)
            return obs
        else:
            return self._transpose(obs)

    def _transpose(self, obs):
        # obs shape is (n_envs, H, W, C) -> transpose to (n_envs, C, H, W)
        return obs.transpose(0, 3, 1, 2)


In [37]:
class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # The original shape remains (84,84,1), but the dtype and range change
        self.observation_space = gym.spaces.Box(
            low=0.0,
            high=1.0,
            shape=self.observation_space.shape,
            dtype=np.float32
        )
        
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


# class ScaledFloatFrame(gym.ObservationWrapper):
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0



class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super().__init__(env)
        # Check that 'FIRE' is a valid action in the environment
        assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
        assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

    def step(self, action):
        return self.env.step(action)

    def reset(self, **kwargs):
        # Reset the environment
        obs, info = self.env.reset(**kwargs)

        # Perform the FIRE action
        obs, _, terminated, truncated, _ = self.env.step(1)
        if terminated or truncated:  # If game ends after FIRE, reset again
            obs, info = self.env.reset(**kwargs)

        return obs, info
        
# Custom wrapper to add channel dimension
class AddChannelDimension(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape
        # Update the observation space to include a channel dimension
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(obs_shape[0], obs_shape[1], 1),
            dtype=np.uint8,
        )

    def observation(self, observation):
        # Add a channel dimension
        return np.expand_dims(observation, axis=-1)

# class MaxAndSkipEnv(gym.Wrapper):
#     def __init__(self, env=None, skip=4):
#         super(MaxAndSkipEnv, self).__init__(env)
#         self._obs_buffer = collections.deque(maxlen=2)
#         self._skip = skip

#     def step(self, action):
#         total_reward = 0.0
#         done = None
#         for _ in range(self._skip):
#             obs, reward, terminated,truncated, info = self.env.step(action)
#             done = terminated or truncated
#             self._obs_buffer.append(obs)
#             total_reward += reward
#             if done:
#                 break
#         max_frame = np.max(np.stack(self._obs_buffer), axis=0)
#         return max_frame, total_reward, terminated, truncated, info

    # def reset(self, *, seed=None, options=None):
    #     self._obs_buffer.clear()
    #     obs, info = self.env.reset(seed=seed, options=options)
    #     self._obs_buffer.append(obs)
    #     return obs, info


def make_env(env_name, obs_type="grayscale", render_mode=None,):
    def _init():
        env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
        print("Standard Env.        : {}".format(env.observation_space.shape))
        env = FireResetEnv(env)
        print("FireResetEnv          : {}".format(env.observation_space.shape))
        # env = MaxAndSkipEnv(env)
        # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
        env = ResizeObservation(env, (84, 84))
        print("ResizeObservation    : {}".format(env.observation_space.shape))
        env = AddChannelDimension(env)  # Add channel dimension here
        print("AddChannelDimension  : {}".format(env.observation_space.shape))
        # env = GrayscaleObservation(env, keep_dim=True)
        # print("GrayscaleObservation : {}".format(env.observation_space.shape))
        
        env = ScaledFloatFrame(env)
        print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        
        # env = Monitor(env, allow_early_resets=False) # from stable baselines
        # print("Monitor               : {}".format(env.observation_space.shape))

        return env
    return _init



# env = DummyVecEnv([make_env(config["env_name"])])
# use make_vec_env with n_envs=16
env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=8)

# stack 4 frames
env = VecFrameStack(env, n_stack=4)
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# convert back to PyTorch format (channel-first)
# env = VecTransposeImage(env)
env = MyVecTransposeImage(env)
print("VecTransposeImage Shape: {}".format(env.observation_space.shape))

print("Final Observation Space: {}".format(env.observation_space.shape))

Standard Env.        : (210, 160)
FireResetEnv          : (210, 160)
ResizeObservation    : (84, 84)
AddChannelDimension  : (84, 84, 1)
ScaledFloatFrame     : (84, 84, 1)
Post VecFrameStack Shape: (84, 84, 4)
VecTransposeImage Shape: (4, 84, 84)
Final Observation Space: (4, 84, 84)


# Start Testing

In [58]:
# class ScaledFloatFrame(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         # The original shape remains (84,84,1), but the dtype and range change
#         self.observation_space = gym.spaces.Box(
#             low=0.0,
#             high=1.0,
#             shape=self.observation_space.shape,
#             dtype=np.float32
#         )
        
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0


# # class ScaledFloatFrame(gym.ObservationWrapper):
# #     def observation(self, obs):
# #         return np.array(obs).astype(np.float32) / 255.0



# class FireResetEnv(gym.Wrapper):
#     def __init__(self, env=None):
#         super().__init__(env)
#         # Check that 'FIRE' is a valid action in the environment
#         assert 'FIRE' in env.unwrapped.get_action_meanings(), "Environment does not support 'FIRE' action"
#         assert len(env.unwrapped.get_action_meanings()) >= 3, "Action space too small for expected actions"

#     def step(self, action):
#         return self.env.step(action)

#     def reset(self, **kwargs):
#         # Reset the environment
#         obs, info = self.env.reset(**kwargs)

#         # Perform the FIRE action
#         obs, _, terminated, truncated, _ = self.env.step(1)
#         if terminated or truncated:  # If game ends after FIRE, reset again
#             obs, info = self.env.reset(**kwargs)

#         return obs, info
        
# # Custom wrapper to add channel dimension
# class AddChannelDimension(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         obs_shape = self.observation_space.shape
#         # Update the observation space to include a channel dimension
#         self.observation_space = gym.spaces.Box(
#             low=0,
#             high=255,
#             shape=(obs_shape[0], obs_shape[1], 1),
#             dtype=np.uint8,
#         )

#     def observation(self, observation):
#         # Add a channel dimension
#         return np.expand_dims(observation, axis=-1)

# # class MaxAndSkipEnv(gym.Wrapper):
# #     def __init__(self, env=None, skip=4):
# #         super(MaxAndSkipEnv, self).__init__(env)
# #         self._obs_buffer = collections.deque(maxlen=2)
# #         self._skip = skip

# #     def step(self, action):
# #         total_reward = 0.0
# #         done = None
# #         for _ in range(self._skip):
# #             obs, reward, terminated,truncated, info = self.env.step(action)
# #             done = terminated or truncated
# #             self._obs_buffer.append(obs)
# #             total_reward += reward
# #             if done:
# #                 break
# #         max_frame = np.max(np.stack(self._obs_buffer), axis=0)
# #         return max_frame, total_reward, terminated, truncated, info

#     # def reset(self, *, seed=None, options=None):
#     #     self._obs_buffer.clear()
#     #     obs, info = self.env.reset(seed=seed, options=options)
#     #     self._obs_buffer.append(obs)
#     #     return obs, info


# def make_env(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
        
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
        
#         # env = Monitor(env, allow_early_resets=False) # from stable baselines
#         # print("Monitor               : {}".format(env.observation_space.shape))

#         return env
#     return _init



# # env = DummyVecEnv([make_env(config["env_name"])])
# # use make_vec_env with n_envs=16
# env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=1) # ALE/MarioBros-v5

# # stack 4 frames
# env = VecFrameStack(env, n_stack=4)
# print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))

# # convert back to PyTorch format (channel-first)
# # env = VecTransposeImage(env)
# env = MyVecTransposeImage(env)
# print("VecTransposeImage Shape: {}".format(env.observation_space.shape))

# print("Final Observation Space: {}".format(env.observation_space.shape))

In [59]:
# obs = env.reset()
# print("Initial observation shape:", obs.shape)
# done = False
# total_reward = 0.0

# for i in range(10000):
#     # Take a random action
#     action = env.action_space.sample()
#     obs, reward, done, info = env.step([action])
#     total_reward += reward

#     # if i % 50 == 0:
#     #     print(f"Step: {i}, Reward: {reward}, Done: {done}, Obs mean: {obs.mean()}")
#     if reward>0:
#         print(f"Step: {i}, Reward: {reward}, Done: {done}, Obs mean: {obs.mean()}")

#     if done:
#         print("Done")
#         obs = env.reset()
#         total_reward = 0.0

In [ ]:
# class ScaledFloatFrame(gym.ObservationWrapper):
#     def __init__(self, env):
#         super().__init__(env)
#         # The original shape remains (84,84,1), but the dtype and range change
#         self.observation_space = gym.spaces.Box(
#             low=0.0,
#             high=1.0,
#             shape=self.observation_space.shape,
#             dtype=np.float32
#         )
        
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0

In [ ]:
# def make_env(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))

#         return env
#     return _init

In [46]:
# # env = gym.make("ALE/Pong-v5", obs_type="grayscale")
# # print("Standard Env.        : {}".format(env.observation_space.shape))
# # env = FireResetEnv(env)
# # print("FireResetEnv          : {}".format(env.observation_space.shape))
# # env = ResizeObservation(env, (84, 84))
# # print("ResizeObservation    : {}".format(env.observation_space.shape))
# # env = AddChannelDimension(env)
# # print("AddChannelDimension   : {}".format(env.observation_space.shape))
# # env = ScaledFloatFrame(env)
# # print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
# # env = Monitor(env, allow_early_resets=False) # from stable baselines
# # print("Monitor               : {}".format(env.observation_space.shape))
# env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=1)
# obs = env.reset()
# print(obs.shape)
# print(obs)
# print("Initial Obs Stats -> min:", obs.min(), "max:", obs.max(), "mean:", obs.mean())

# End Testing

In [ ]:
print("Check")
print("Post VecFrameStack Shape: {}".format(env.observation_space.shape))
print("Final Observation Space: {}".format(env.observation_space.shape))

In [ ]:
# def make_env_evual(env_name, obs_type="grayscale", render_mode=None,):
#     def _init():
#         env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
#         # env = Monitor(env, allow_early_resets=False) # from stable baselines
#         # print("Monitor               : {}".format(env.observation_space.shape))

#         return env
#     return _init


# Create an evaluation environment (similar to our training env)
eval_env = make_vec_env(env_id=make_env(config["env_name"]), n_envs=1)

# stack 4 frames
eval_env = VecFrameStack(eval_env, n_stack=4)
print("eval_env Post VecFrameStack Shape: {}".format(eval_env.observation_space.shape))

# convert back to PyTorch format (channel-first)
# eval_env = VecTransposeImage(eval_env)
eval_env = MyVecTransposeImage(eval_env)
print("eval_env MyVecTransposeImage Shape: {}".format(eval_env.observation_space.shape))
print("eval_env Final Observation Space: {}".format(eval_env.observation_space.shape))

In [ ]:
# obs = eval_env.reset()
# print(obs.shape)
# print(obs)
# print("Initial Obs Stats -> min:", obs.min(), "max:", obs.max(), "mean:", obs.mean())

In [ ]:
# define
# batch_size=30048,
# policy_kwargs = dict(net_arch=[512, 256])
model = PPO(config["policy_type"], 
            env, 
            verbose=0, 
            tensorboard_log=f"runs/{run.id}",
            batch_size=256, 
            learning_rate=2.5e-4, #2.5e-4, 0.001
            gamma=0.99,
            n_steps=128,
            n_epochs=4,
            clip_range=0.1,
            vf_coef=0.5,
            ent_coef=0.05, # 0.01
            policy_kwargs=dict(net_arch=[512, 256], normalize_images=False),
            device="cuda")


In [ ]:
# Create the evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=config["export_path"],  # directory to save the best model
    log_path=config["export_path"],              # evaluation logs
    eval_freq=100000,                            # evaluate the model every 100,000 steps
    deterministic=True,
    render=False
)

In [ ]:
class GradientInspectionCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(GradientInspectionCallback, self).__init__(verbose)
    
    def _on_step(self) -> bool:
        # Access the policy network
        policy_net = self.model.policy
        
        # Iterate over the parameters to inspect gradients
        for name, param in policy_net.named_parameters():
            if param.grad is not None:
                grad_norm = param.grad.norm().item()
                self.logger.record(f"gradients/{name}_norm", grad_norm)
        
        return True  # Continue training


In [ ]:
class DebugObservationCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(DebugObservationCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Get the current observation
        observation = self.locals["new_obs"]  # Observations from the environment
        print("Observation mean:", np.mean(observation))
        print("Observation std:", np.std(observation))
        print("Observation min:", np.min(observation))
        print("Observation max:", np.max(observation))

        # Stop training if all observations are zero (optional)
        # if np.all(observation == 0):
        #     print("All observation values are zero. Exiting...")
        #     return False

        return True  # Continue training

In [ ]:
# Combine both callbacks
# callbacks = CallbackList([eval_callback, WandbCallback(verbose=2), GradientInspectionCallback(), DebugObservationCallback()])
callbacks = CallbackList([eval_callback, WandbCallback(verbose=2), GradientInspectionCallback()])

In [ ]:
# train
t0 = datetime.now()
model.learn(total_timesteps=config["total_timesteps"], callback=callbacks)
t1 = datetime.now()
print('>>> Training time (hh:mm:ss.ms): {}'.format(t1-t0))

# save and export model
model.save(config["export_path"] + config["model_name"])

Training time without parallezing env: >>> Training time (hh:mm:ss.ms): 1:01:58.577332

In [ ]:
print("Finish")

In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/exports")
# os.remove("/kaggle/working/Breakout-v5.zip")

In [ ]:
# !zip -r folder_name.zip exports
# from google.colab import files
# files.download('folder_name.zip')

In [ ]:
# env = gym.make(env_name, obs_type="grayscale", render_mode=render_mode)
#         print("Standard Env.        : {}".format(env.observation_space.shape))
#         env = FireResetEnv(env)
#         print("FireResetEnv          : {}".format(env.observation_space.shape))
#         # env = MaxAndSkipEnv(env)
#         # print("MaxAndSkipEnv        : {}".format(env.observation_space.shape))
#         env = ResizeObservation(env, (84, 84))
#         print("ResizeObservation    : {}".format(env.observation_space.shape))
#         env = AddChannelDimension(env)  # Add channel dimension here
#         print("AddChannelDimension  : {}".format(env.observation_space.shape))
#         # env = GrayscaleObservation(env, keep_dim=True)
#         # print("GrayscaleObservation : {}".format(env.observation_space.shape))
#         env = ScaledFloatFrame(env)
#         print("ScaledFloatFrame     : {}".format(env.observation_space.shape))
#         env = Monitor(env, allow_early_resets=False) # from stable baselines
#         print("Monitor 